In [1]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor,DistanceCalculator
import json
from Bio import SeqIO

import subprocess

In [4]:
input_file = "24varseq.fasta"
output_file = "24varseqoutput.fasta"

# Define the MUSCLE command as a list of arguments
muscle_command = ["muscle5.1.win64.exe", "-align", input_file, "-output", output_file]

# Run the MUSCLE command using subprocess
process = subprocess.Popen(muscle_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Get the standard output and error messages
stdout, stderr = process.communicate()

# Print the standard output and error messages
print(stdout.decode())
print(stderr.decode())

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [9]:
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator,DistanceTreeConstructor
from Bio import Phylo

alignment =AlignIO.read("24varseqoutput.fasta","fasta")

calculator=DistanceCalculator("identity")
dm=calculator.get_distance(alignment)

constructor=DistanceTreeConstructor(calculator,"upgma")
tree=constructor.build_tree(alignment)

In [10]:
Phylo.write(tree,"tree.nwk","newick")

1

In [11]:
def get_tuple(clade):
    if clade.is_terminal():
        return clade.name
    else:
        return (get_tuple(clade.clades[0]), get_tuple(clade.clades[1]))

tree_tuple = get_tuple(tree.clade)

print(tree_tuple)

((((('W159', 'W173'), (('W294', ('W225', 'W118')), ('C029', ('C014', 'C152')))), (((('C195', (((('C054', 'C034'), 'W068'), (('W056', 'W059'), 'W281')), 'W290')), (('W207', (('W120', ('W327', ('C012', (((('C064', 'W027'), 'C048'), 'W209'), 'W052')))), ('W160', 'W109'))), ('W158', ('C051', 'C119')))), (('C179', 'W122'), (('C010', ('W062', 'W041')), ((('W288', 'W007'), (('C153', 'W022'), ('W123', 'W008'))), (('W302', (('W329', 'W260'), ('W102', 'W273'))), (('C123', 'W251'), ('W257', (('W254', 'W017'), ((((((((('W253', 'W275'), 'W024'), 'W005'), 'W113'), 'W303'), 'W053'), 'W065'), 'W057'), 'W006'))))))))), (('W125', ('C032', 'W038')), ('W128', (('W119', ('W226', ('W193', (('W010', 'W063'), 'W061')))), ((('C023', 'W045'), ('C035', (('C057', 'C017'), 'C177'))), ((('C063', (('C172', 'C120'), 'C149')), ((('C085', ('C016', 'C056')), (((('W139', 'W064'), 'W018'), 'W019'), ('W127', 'W326'))), ('W011', 'C070'))), ((((('W324', 'W323'), 'C084'), (('W021', 'C185'), 'C186')), 'W003'), (('W013', 'W328'

In [12]:
s = str(tree_tuple).replace(" ", "").replace("'", "")
non_characters = ["(", ")", "'"]
clusters = []
i = 0
while i < len(s):
    if s[i] == "(":
        clusterOpen = i
        flag = 0
        i+=1
        while i < len(s):
            if s[i] not in non_characters:
                flag = 1
            if s[i] == "(" and flag == 1:
                break
            i+=1
        clusterClose = i-1
        cluster = s[clusterOpen:clusterClose].replace("(", "").replace(")", "").split(",")
        clusters.append(cluster)

clusters
len(clusters)

221

In [13]:
import pandas as pd 
import numpy as np

In [15]:
df=pd.read_csv("merged_final24.csv")
df

,Sequence,Subpopulation,Cultivar ID,Plant Height (cm)
0,GAAAGCCGTATGTGTACCAACCTT,Temperate Japonica,C152,100.00
1,GGGAGCGACAGGTATGTTACCTTT,Indica I,W146,127.40
2,GGGAGCGGTAGTAATGTTACCTTT,Indica II,W150,104.60
3,GGGGGCCACCGTAGTACCACCCTC,VI/Aromatic,W129,151.20
4,GGGGGCGGTATGTATGTTACCTTT,Indica II,W086,103.40
...,...,...,...,...
524,GGGAACGGTATTAATGTTACCTCT,Indica I,C159,95.80
525,GGGAGCGGTATGTATGTTACCTTT,Indica Intermediate,W167,135.20
526,GAGAGCCATAGGTATACCAACCTC,Japonica Intermediate,W324,156.00
527,GGGGACCACCGTAATGTTTCCTTC,Indica Intermediate,C198,173.77


In [16]:
d = dict()

for i in clusters:
    df1 = df[df["Cultivar ID"].isin(i)]
    value_counts = df1["Subpopulation"].value_counts()
    most_frequent_value = value_counts.idxmax()
    df2 = df1[df1["Subpopulation"].isin([most_frequent_value])]
    minheight = min(df2["Plant Height (cm)"])
    maxheight = max(df2["Plant Height (cm)"])
    
    for j in i:
        d1 = dict()
        d1["sequence"] = df[df["Cultivar ID"]==j]["Sequence"].iloc[0]
        d1["Subpopulation"]=most_frequent_value
        d1["minheight"]=minheight
        d1["maxheight"]=maxheight
        d1["height_range"]=str(minheight)+" - "+str(maxheight)
        d[j] = d1
d

{'W159': {'sequence': 'GGGAGCGGTAGGTATACCACCTTT',
  'Subpopulation': 'Tropical Japonica',
  'minheight': 121.6,
  'maxheight': 121.6,
  'height_range': '121.6 - 121.6'},
 'W173': {'sequence': 'AGGAGCCATAGGAATACCACCCTT',
  'Subpopulation': 'Tropical Japonica',
  'minheight': 121.6,
  'maxheight': 121.6,
  'height_range': '121.6 - 121.6'},
 'W294': {'sequence': 'GGGGGCCGTATTAATGCCAACTTT',
  'Subpopulation': 'Intermediate',
  'minheight': 118.8,
  'maxheight': 118.8,
  'height_range': '118.8 - 118.8'},
 'W225': {'sequence': 'GAGGGCGGTATGAATGCCAACTTT',
  'Subpopulation': 'Tropical Japonica',
  'minheight': 84.2,
  'maxheight': 90.2,
  'height_range': '84.2 - 90.2'},
 'W118': {'sequence': 'GAGGGCGGTATGTGTGCCAACTTT',
  'Subpopulation': 'Tropical Japonica',
  'minheight': 84.2,
  'maxheight': 90.2,
  'height_range': '84.2 - 90.2'},
 'C029': {'sequence': 'GGAAGCCGTATGTATACCAACCTT',
  'Subpopulation': 'Japonica Intermediate',
  'minheight': 94.0,
  'maxheight': 94.0,
  'height_range': '94.0 - 9

In [6]:
import json

In [19]:
with open('final_classification.json', 'w') as f:
    json.dump(d, f)

In [7]:
with open('final_classification.json', 'r') as f:
    data = json.load(f)

In [13]:
def makequery(query,name="sequence"):
    with open('query.fasta', 'w') as f:
        f.write(">{}\n".format(name))
        f.write("{}".format(query))
        


In [18]:
name=input("Please enter the name")
query=input("please enter a sequence")
makequery(query,name)

In [20]:
import subprocess


command = ['magicblast', '-query', 'query.fasta', '-subject', '24varseq.fasta', '-out', 'o1.txt']


subprocess.run(command)

CompletedProcess(args=['magicblast', '-query', 'query.fasta', '-subject', '24varseq.fasta', '-out', 'o1.txt'], returncode=0)

In [22]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
query_file = "query.fasta"

# Define the path to the FASTA file containing the database of sequences to search
database_file = "24varseq.fasta"

# Read the query sequence from the query FASTA file
with open(query_file, "r") as f:
    query_record = SeqIO.read(f, "fasta")
query_sequence = query_record.seq

# Search for similar sequences using BLAST
result_handle = NCBIWWW.qblast("blastn", "nt", query_sequence)

# Parse the BLAST results
blast_record = NCBIXML.read(result_handle)

# Extract the top hit sequence ID and description
top_hit = blast_record.alignments[0]
top_hit_id = top_hit.hit_id
top_hit_description = top_hit.hit_def

# Read the top hit sequence from the database FASTA file
with open(database_file, "r") as f:
    for record in SeqIO.parse(f, "fasta"):
        if record.id == top_hit_id:
            top_hit_sequence = record.seq
            break

# Print the top hit sequence ID, description, and sequence
print("Top hit ID: ", top_hit_id)
print("Top hit description: ", top_hit_description)
print("Top hit sequence: ", top_hit_sequence)

Top hit ID:  gi|1922864589|gb|CP062047.1|
Top hit description:  Macrobrachium nipponense isolate FS-2020 chromosome 34


NameError: name 'top_hit_sequence' is not defined